QKD - BB84 Automated (with optional Eve)


Install necessary libraries (quietly)


In [ ]:
%pip install cirq --quiet
import random
import cirq


Generate random bits and bases


In [ ]:
# Parameters
num_qubits = 16  # you can change this number to any positive integer

# Only user option: enable/disable Eve
include_eve = input("Include Eve? (Y/N): ").strip().upper()
if include_eve not in ("Y", "N"):
    raise ValueError("Enter Y or N for including Eve.")

# Randomly generate Alice's bits and bases
alice_bits = [random.choice([0, 1]) for _ in range(num_qubits)]
alice_bases = [random.choice(["R", "D"]) for _ in range(num_qubits)]

# Randomly generate Bob's bases
bob_bases = [random.choice(["R", "D"]) for _ in range(num_qubits)]

eve_bases = [random.choice(["R", "D"]) for _ in range(num_qubits)] if include_eve == "Y" else None

print("Phase 1 - Sending")
print("Alice bits:", alice_bits)
print("Alice bases:", alice_bases)
print("Bob bases:", bob_bases)
if include_eve == "Y":
    print("Eve bases:", eve_bases)


Simulate Alice’s encoding and Bob’s measurement


In [ ]:
# Build qubits and circuit
qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
circuit = cirq.Circuit()

# Alice encodes bits
for i in range(num_qubits):
    if alice_bits[i] == 1:
        circuit.append(cirq.X(qubits[i]))
    if alice_bases[i] == 'D':
        circuit.append(cirq.H(qubits[i]))

# Optional Eve intercept-resend
if include_eve == 'Y':
    for i in range(num_qubits):
        if eve_bases[i] == 'D':
            circuit.append(cirq.H(qubits[i]))
        circuit.append(cirq.measure(qubits[i], key=f'e{i}'))
        if eve_bases[i] == 'D':
            circuit.append(cirq.H(qubits[i]))

print("Phase 2 - Receiving")

# Bob rotates to his basis then measures
for i in range(num_qubits):
    if bob_bases[i] == 'D':
        circuit.append(cirq.H(qubits[i]))

circuit.append(cirq.measure(*qubits, key='bob'))

sim = cirq.Simulator()
res = sim.run(circuit, repetitions=1)

bob_bits = [int(res.measurements['bob'][0][i]) for i in range(num_qubits)]

print("Bob bits:", bob_bits)


(Optional) Simulate Eve’s interference


Compare keys and show results


In [ ]:
print("Phase 3 - Comparing")

# Sift keys where bases match
alice_key = []
bob_key = []
match_positions = []
for i in range(num_qubits):
    if alice_bases[i] == bob_bases[i]:
        alice_key.append(alice_bits[i])
        bob_key.append(bob_bits[i])
        match_positions.append(i)

matched = len(match_positions)
errors = sum(1 for a, b in zip(alice_key, bob_key) if a != b)
qber = (errors / matched) * 100 if matched else 0.0

print("Matched positions:", matched)
print("Alice key:", alice_key)
print("Bob key:  ", bob_key)
print(f"QBER: {qber:.2f}%")

if include_eve == 'Y':
    print("Eve was enabled. Higher QBER indicates eavesdropping.")
else:
    print("Eve was disabled. Keys should correlate closely (QBER ~ 0%).")
